In [10]:
# Démarrage de tensorboard pour notebook
%load_ext tensorboard

# Les imports majeurs, certains ne sont peut être plus utiles -> à re-vérifier 
import sys
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, LSTM
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
import datetime
import os
import pandas as pd
#from PIL import Image
import shutil  

from tensorflow.python.framework import ops #pour tenter de reset tensorboard, sans grand succès
ops.reset_default_graph()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
# Chargement des datasets de train et de validation + one hot encoding
def load_dataset():
    # Chargement des données cifar10
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode encoding sur les labels
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

# Normalisation pour accroître la vitesse du modèle (en redimensionnant les pixels)
def prep_pixels(train, test):
    # Convertion des int en float
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # Normalisation pour avoir des nombres entre 0 et 1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # Retourner les images normalisées
    return train_norm, test_norm

In [12]:
mnist= tensorflow.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

x_train=x_train/255.0
y_train=y_train/255.0
x_test=x_test/255.0
y_test=y_test/255.0

x_train.shape[1:]

NameError: name 'tensorflow' is not defined

In [7]:
def load_dataset():
    # Chargement des données cifar10
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode encoding sur les labels
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)  
    
    Xtrain = np.reshape(np.asarray([np.mean(im, axis=2, keepdims=True) for im in trainX]),(50000,32,32))
    Xtest = np.reshape(np.asarray([np.mean(im, axis=2, keepdims=True) for im in testX]),(10000,32,32))
        
    
    return Xtrain, trainY, Xtest, testY

In [8]:
x_train, y_train, x_test, y_test=load_dataset()

In [167]:
x_train.shape

(50000, 32, 32)

In [169]:
model = Sequential()

#model.add(Flatten(input_shape= (32,32)))

model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(128, activation='tanh'))
model.add(Dropout(0.1))

model.add(Dense(10, activation='softmax'))


In [173]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs=5,
          validation_data=(x_test, y_test),
         batch_size=32)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 117s 2ms/sample - loss: 2.3140 - accuracy: 0.0995 - val_loss: 2.3252 - val_accuracy: 0.1000
Epoch 2/5
50000/50000 [==============================] - 114s 2ms/sample - loss: 2.3152 - accuracy: 0.1003 - val_loss: 2.3199 - val_accuracy: 0.1002
Epoch 3/5
50000/50000 [==============================] - 107s 2ms/sample - loss: 2.3123 - accuracy: 0.0991 - val_loss: 2.3054 - val_accuracy: 0.1000
Epoch 4/5
50000/50000 [==============================] - 112s 2ms/sample - loss: 2.3128 - accuracy: 0.0994 - val_loss: 2.3110 - val_accuracy: 0.1000
Epoch 5/5
50000/50000 [==============================] - 108s 2ms/sample - loss: 2.3129 - accuracy: 0.0987 - val_loss: 2.3118 - val_accuracy: 0.1000


In [174]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128, activation='relu', return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, activation='relu', return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, activation='tanh'))
rnn.add(Dropout(0.1))

rnn.add(Dense(10, activation='softmax'))

In [175]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

rnn.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=5,
          validation_data=(x_test, y_test),
         batch_size=32)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 158s 3ms/sample - loss: 2.3619 - accuracy: 0.1042 - val_loss: 2.3248 - val_accuracy: 0.0930
Epoch 2/5
50000/50000 [==============================] - 142s 3ms/sample - loss: 2.3091 - accuracy: 0.1106 - val_loss: 2.2919 - val_accuracy: 0.1246
Epoch 3/5
50000/50000 [==============================] - 146s 3ms/sample - loss: 2.3095 - accuracy: 0.1065 - val_loss: 2.3184 - val_accuracy: 0.0832
Epoch 4/5
50000/50000 [==============================] - 139s 3ms/sample - loss: 2.3101 - accuracy: 0.1033 - val_loss: 2.2842 - val_accuracy: 0.1346
Epoch 5/5
50000/50000 [==============================] - 139s 3ms/sample - loss: 2.2829 - accuracy: 0.1225 - val_loss: 2.2510 - val_accuracy: 0.1317


In [176]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

rnn.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=5,
          validation_data=(x_test, y_test),
         batch_size=12)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 384s 8ms/sample - loss: 2.3253 - accuracy: 0.1010 - val_loss: 2.3127 - val_accuracy: 0.1024
Epoch 2/5
50000/50000 [==============================] - 351s 7ms/sample - loss: 2.3246 - accuracy: 0.1015 - val_loss: 2.3140 - val_accuracy: 0.1000
Epoch 3/5
50000/50000 [==============================] - 354s 7ms/sample - loss: 2.3256 - accuracy: 0.0980 - val_loss: 2.3090 - val_accuracy: 0.1000
Epoch 4/5
50000/50000 [==============================] - 344s 7ms/sample - loss: 2.3233 - accuracy: 0.1005 - val_loss: 2.3214 - val_accuracy: 0.0955
Epoch 5/5
50000/50000 [==============================] - 432s 9ms/sample - loss: 2.3225 - accuracy: 0.1001 - val_loss: 2.3262 - val_accuracy: 0.1217


In [178]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-4)

rnn.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=5,
          validation_data=(x_test, y_test),
         batch_size=32)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 133s 3ms/sample - loss: 2.3157 - accuracy: 0.1002 - val_loss: 2.3084 - val_accuracy: 0.1036
Epoch 2/5
50000/50000 [==============================] - 138s 3ms/sample - loss: 2.3138 - accuracy: 0.0967 - val_loss: 2.3191 - val_accuracy: 0.1000
Epoch 3/5
50000/50000 [==============================] - 132s 3ms/sample - loss: 2.3127 - accuracy: 0.0998 - val_loss: 2.3063 - val_accuracy: 0.0998
Epoch 4/5
50000/50000 [==============================] - 141s 3ms/sample - loss: 2.3113 - accuracy: 0.1022 - val_loss: 2.3113 - val_accuracy: 0.1000
Epoch 5/5
50000/50000 [==============================] - 119s 2ms/sample - loss: 2.3125 - accuracy: 0.0983 - val_loss: 2.3120 - val_accuracy: 0.0966


In [179]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-4)

rnn.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=5,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 47s 933us/sample - loss: 2.3106 - accuracy: 0.1019 - val_loss: 2.3154 - val_accuracy: 0.1004
Epoch 2/5
50000/50000 [==============================] - 42s 847us/sample - loss: 2.3120 - accuracy: 0.0988 - val_loss: 2.3111 - val_accuracy: 0.1004
Epoch 3/5
50000/50000 [==============================] - 43s 870us/sample - loss: 2.3099 - accuracy: 0.0997 - val_loss: 2.3120 - val_accuracy: 0.0999
Epoch 4/5
50000/50000 [==============================] - 41s 812us/sample - loss: 2.3108 - accuracy: 0.0992 - val_loss: 2.3205 - val_accuracy: 0.1000
Epoch 5/5
50000/50000 [==============================] - 41s 820us/sample - loss: 2.3105 - accuracy: 0.1010 - val_loss: 2.3111 - val_accuracy: 0.1000


In [182]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-4)

rnn.compile(
    loss='mean_squared_error',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=20,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 61s 1ms/sample - loss: 0.0902 - accuracy: 0.0987 - val_loss: 0.0902 - val_accuracy: 0.0999
Epoch 2/20
50000/50000 [==============================] - 43s 863us/sample - loss: 0.0901 - accuracy: 0.1007 - val_loss: 0.0900 - val_accuracy: 0.1000
Epoch 3/20
50000/50000 [==============================] - 44s 873us/sample - loss: 0.0901 - accuracy: 0.1016 - val_loss: 0.0902 - val_accuracy: 0.0868
Epoch 4/20
50000/50000 [==============================] - 40s 805us/sample - loss: 0.0901 - accuracy: 0.1015 - val_loss: 0.0901 - val_accuracy: 0.1000
Epoch 5/20
50000/50000 [==============================] - 41s 813us/sample - loss: 0.0901 - accuracy: 0.0992 - val_loss: 0.0902 - val_accuracy: 0.1000
Epoch 6/20
50000/50000 [==============================] - 41s 819us/sample - loss: 0.0901 - accuracy: 0.0996 - val_loss: 0.0901 - val_accuracy: 0.1000
Epoch 7/20
50000/50000 [======================

In [183]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128))
rnn.add(Dropout(0.1))

rnn.add(Dense(10))

In [184]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-4)

rnn.compile(
    loss='mean_squared_error',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=20,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 18s 362us/sample - loss: 0.0849 - accuracy: 0.2545 - val_loss: 0.0802 - val_accuracy: 0.3279
Epoch 2/20
50000/50000 [==============================] - 45s 894us/sample - loss: 0.0801 - accuracy: 0.3222 - val_loss: 0.0782 - val_accuracy: 0.3358
Epoch 3/20
50000/50000 [==============================] - 15s 300us/sample - loss: 0.0777 - accuracy: 0.3528 - val_loss: 0.0759 - val_accuracy: 0.3624
Epoch 4/20
50000/50000 [==============================] - 14s 282us/sample - loss: 0.0754 - accuracy: 0.3796 - val_loss: 0.0737 - val_accuracy: 0.3896
Epoch 5/20
50000/50000 [==============================] - 14s 282us/sample - loss: 0.0732 - accuracy: 0.4049 - val_loss: 0.0712 - val_accuracy: 0.4251
Epoch 6/20
50000/50000 [==============================] - 14s 280us/sample - loss: 0.0714 - accuracy: 0.4267 - val_loss: 0.0693 - val_accuracy: 0.4399
Epoch 7/20
50000/50000 [====================

In [185]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128))
rnn.add(Dropout(0.1))

rnn.add(Dense(10))

In [186]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-4)

rnn.compile(
    loss='mean_squared_error',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=20,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 51s 1ms/sample - loss: 0.0833 - accuracy: 0.2771 - val_loss: 0.0808 - val_accuracy: 0.3093
Epoch 2/20
50000/50000 [==============================] - 24s 473us/sample - loss: 0.0795 - accuracy: 0.3292 - val_loss: 0.0768 - val_accuracy: 0.3536
Epoch 3/20
50000/50000 [==============================] - 23s 469us/sample - loss: 0.0767 - accuracy: 0.3598 - val_loss: 0.0798 - val_accuracy: 0.3349
Epoch 4/20
50000/50000 [==============================] - 24s 472us/sample - loss: 0.0739 - accuracy: 0.3927 - val_loss: 0.0718 - val_accuracy: 0.4143
Epoch 5/20
50000/50000 [==============================] - 24s 479us/sample - loss: 0.0715 - accuracy: 0.4196 - val_loss: 0.0687 - val_accuracy: 0.4485
Epoch 6/20
50000/50000 [==============================] - 24s 478us/sample - loss: 0.0693 - accuracy: 0.4438 - val_loss: 0.0673 - val_accuracy: 0.4615
Epoch 7/20
50000/50000 [======================

In [189]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128))
rnn.add(Dropout(0.1))

rnn.add(Dense(10))

In [190]:
rnn.compile(
    loss='mean_squared_error',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=20,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 20s 400us/sample - loss: 0.0852 - accuracy: 0.2500 - val_loss: 0.0806 - val_accuracy: 0.3140
Epoch 2/20
50000/50000 [==============================] - 15s 306us/sample - loss: 0.0801 - accuracy: 0.3190 - val_loss: 0.0772 - val_accuracy: 0.3542
Epoch 3/20
50000/50000 [==============================] - 15s 305us/sample - loss: 0.0775 - accuracy: 0.3570 - val_loss: 0.0760 - val_accuracy: 0.3616
Epoch 4/20
50000/50000 [==============================] - 15s 304us/sample - loss: 0.0748 - accuracy: 0.3910 - val_loss: 0.0728 - val_accuracy: 0.3991
Epoch 5/20
50000/50000 [==============================] - 15s 304us/sample - loss: 0.0726 - accuracy: 0.4136 - val_loss: 0.0713 - val_accuracy: 0.4177
Epoch 6/20
50000/50000 [==============================] - 15s 302us/sample - loss: 0.0708 - accuracy: 0.4327 - val_loss: 0.0686 - val_accuracy: 0.4487
Epoch 7/20
50000/50000 [====================

In [193]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128))
rnn.add(Dropout(0.1))

rnn.add(Dense(10))

In [192]:
rnn.compile(
    loss='mean_squared_error',
    optimizer=opt,
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=20,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 18s 369us/sample - loss: 0.0881 - accuracy: 0.2154 - val_loss: 0.0832 - val_accuracy: 0.2838
Epoch 2/20
50000/50000 [==============================] - 11s 220us/sample - loss: 0.0826 - accuracy: 0.2891 - val_loss: 0.0801 - val_accuracy: 0.3146
Epoch 3/20
50000/50000 [==============================] - 11s 227us/sample - loss: 0.0803 - accuracy: 0.3237 - val_loss: 0.0781 - val_accuracy: 0.3421
Epoch 4/20
50000/50000 [==============================] - 11s 218us/sample - loss: 0.0785 - accuracy: 0.3434 - val_loss: 0.0770 - val_accuracy: 0.3483
Epoch 5/20
50000/50000 [==============================] - 10s 208us/sample - loss: 0.0769 - accuracy: 0.3661 - val_loss: 0.0756 - val_accuracy: 0.3719
Epoch 6/20
50000/50000 [==============================] - 11s 213us/sample - loss: 0.0754 - accuracy: 0.3839 - val_loss: 0.0740 - val_accuracy: 0.3905
Epoch 7/20
50000/50000 [====================

In [194]:
rnn.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=20,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 29s 589us/sample - loss: 0.0852 - accuracy: 0.2544 - val_loss: 0.0812 - val_accuracy: 0.3069
Epoch 2/20
50000/50000 [==============================] - 15s 301us/sample - loss: 0.0804 - accuracy: 0.3147 - val_loss: 0.0784 - val_accuracy: 0.3368
Epoch 3/20
50000/50000 [==============================] - 15s 307us/sample - loss: 0.0780 - accuracy: 0.3475 - val_loss: 0.0756 - val_accuracy: 0.3692
Epoch 4/20
50000/50000 [==============================] - 15s 308us/sample - loss: 0.0759 - accuracy: 0.3722 - val_loss: 0.0752 - val_accuracy: 0.3836
Epoch 5/20
50000/50000 [==============================] - 15s 298us/sample - loss: 0.0742 - accuracy: 0.3921 - val_loss: 0.0716 - val_accuracy: 0.4181
Epoch 6/20
50000/50000 [==============================] - 15s 298us/sample - loss: 0.0723 - accuracy: 0.4131 - val_loss: 0.0706 - val_accuracy: 0.4294
Epoch 7/20
50000/50000 [====================

In [195]:
rnn.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=50,
          validation_data=(x_test, y_test),
         batch_size=90)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 30s 593us/sample - loss: 0.0547 - accuracy: 0.5859 - val_loss: 0.0596 - val_accuracy: 0.5422
Epoch 2/50
50000/50000 [==============================] - 17s 346us/sample - loss: 0.0539 - accuracy: 0.5943 - val_loss: 0.0587 - val_accuracy: 0.5489
Epoch 3/50
50000/50000 [==============================] - 18s 361us/sample - loss: 0.0530 - accuracy: 0.6025 - val_loss: 0.0589 - val_accuracy: 0.5506
Epoch 4/50
50000/50000 [==============================] - 18s 353us/sample - loss: 0.0521 - accuracy: 0.6115 - val_loss: 0.0585 - val_accuracy: 0.5518
Epoch 5/50
50000/50000 [==============================] - 18s 356us/sample - loss: 0.0524 - accuracy: 0.6093 - val_loss: 0.0587 - val_accuracy: 0.5506
Epoch 6/50
50000/50000 [==============================] - 18s 365us/sample - loss: 0.0512 - accuracy: 0.6197 - val_loss: 0.0595 - val_accuracy: 0.5488
Epoch 7/50
50000/50000 [====================

In [196]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128))
rnn.add(Dropout(0.1))

rnn.add(Dense(10))

In [197]:
rnn.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=50,
          validation_data=(x_test, y_test),
         batch_size=90)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 43s 860us/sample - loss: 0.0839 - accuracy: 0.2683 - val_loss: 0.0842 - val_accuracy: 0.2531
Epoch 2/50
50000/50000 [==============================] - 23s 452us/sample - loss: 0.0802 - accuracy: 0.3186 - val_loss: 0.0783 - val_accuracy: 0.3354
Epoch 3/50
50000/50000 [==============================] - 23s 452us/sample - loss: 0.0781 - accuracy: 0.3419 - val_loss: 0.0766 - val_accuracy: 0.3535
Epoch 4/50
50000/50000 [==============================] - 22s 440us/sample - loss: 0.0764 - accuracy: 0.3651 - val_loss: 0.0737 - val_accuracy: 0.3866
Epoch 5/50
50000/50000 [==============================] - 23s 451us/sample - loss: 0.0741 - accuracy: 0.3919 - val_loss: 0.0728 - val_accuracy: 0.3971
Epoch 6/50
50000/50000 [==============================] - 22s 443us/sample - loss: 0.0720 - accuracy: 0.4142 - val_loss: 0.0702 - val_accuracy: 0.4287
Epoch 7/50
50000/50000 [====================

In [199]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))


rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128))
rnn.add(Dropout(0.1))

rnn.add(Dense(10))

In [200]:
rnn.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=50,
          validation_data=(x_test, y_test),
         batch_size=90)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 48s 967us/sample - loss: 0.0835 - accuracy: 0.2707 - val_loss: 0.0800 - val_accuracy: 0.3164
Epoch 2/50
50000/50000 [==============================] - 26s 522us/sample - loss: 0.0798 - accuracy: 0.3199 - val_loss: 0.0768 - val_accuracy: 0.3538
Epoch 3/50
50000/50000 [==============================] - 33s 658us/sample - loss: 0.0780 - accuracy: 0.3431 - val_loss: 0.0771 - val_accuracy: 0.3508
Epoch 4/50
50000/50000 [==============================] - 35s 693us/sample - loss: 0.0761 - accuracy: 0.3691 - val_loss: 0.0747 - val_accuracy: 0.3809
Epoch 5/50
50000/50000 [==============================] - 35s 696us/sample - loss: 0.0741 - accuracy: 0.3899 - val_loss: 0.0721 - val_accuracy: 0.4084
Epoch 6/50
50000/50000 [==============================] - 36s 723us/sample - loss: 0.0720 - accuracy: 0.4162 - val_loss: 0.0693 - val_accuracy: 0.4338
Epoch 7/50
50000/50000 [====================

In [13]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))


rnn.add(LSTM(128, return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(128))
rnn.add(Dropout(0.1))

rnn.add(Dense(10))

In [14]:
rnn.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=70,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/70
50000/50000 [==============================] - 42s 831us/sample - loss: 0.0835 - accuracy: 0.2738 - val_loss: 0.0817 - val_accuracy: 0.3061
Epoch 2/70
50000/50000 [==============================] - 22s 448us/sample - loss: 0.0797 - accuracy: 0.3238 - val_loss: 0.0769 - val_accuracy: 0.3550
Epoch 3/70
50000/50000 [==============================] - 21s 422us/sample - loss: 0.0767 - accuracy: 0.3616 - val_loss: 0.0739 - val_accuracy: 0.3832
Epoch 4/70
50000/50000 [==============================] - 21s 425us/sample - loss: 0.0744 - accuracy: 0.3878 - val_loss: 0.0723 - val_accuracy: 0.4079
Epoch 5/70
50000/50000 [==============================] - 22s 430us/sample - loss: 0.0724 - accuracy: 0.4088 - val_loss: 0.0711 - val_accuracy: 0.4198
Epoch 6/70
50000/50000 [==============================] - 22s 437us/sample - loss: 0.0699 - accuracy: 0.4363 - val_loss: 0.0687 - val_accuracy: 0.4447
Epoch 7/70
50000/50000 [====================

In [15]:
rnn = Sequential()

#model.add(Flatten(input_shape= (32,32)))

rnn.add(LSTM(128))
rnn.add(Dropout(0.1))

rnn.add(Dense(10))

In [16]:
rnn.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy'],
)

rnn.fit(x_train,
          y_train,
          epochs=70,
          validation_data=(x_test, y_test),
         batch_size=100)

Train on 50000 samples, validate on 10000 samples
Epoch 1/70
50000/50000 [==============================] - 6s 128us/sample - loss: 0.1084 - accuracy: 0.1070 - val_loss: 0.0914 - val_accuracy: 0.1169
Epoch 2/70
50000/50000 [==============================] - 4s 83us/sample - loss: 0.0913 - accuracy: 0.1328 - val_loss: 0.0889 - val_accuracy: 0.1564
Epoch 3/70
50000/50000 [==============================] - 4s 83us/sample - loss: 0.0895 - accuracy: 0.1397 - val_loss: 0.0900 - val_accuracy: 0.1200
Epoch 4/70
50000/50000 [==============================] - 6s 121us/sample - loss: 0.0893 - accuracy: 0.1387 - val_loss: 0.0888 - val_accuracy: 0.1509
Epoch 5/70
50000/50000 [==============================] - 13s 266us/sample - loss: 0.0888 - accuracy: 0.1550 - val_loss: 0.0883 - val_accuracy: 0.1669
Epoch 6/70
50000/50000 [==============================] - 17s 337us/sample - loss: 0.0886 - accuracy: 0.1601 - val_loss: 0.0886 - val_accuracy: 0.1656
Epoch 7/70
50000/50000 [==========================